In [345]:
#!/usr/bin/env python
import tensornetwork as tn
import numpy as np
import scipy as sp

In [346]:
#создать рандомный массив для будущей MPS
def genrandarr():
    num=[]
    num.append(np.random.rand(p,D))
    for i in range(1,N-1):
        num.append(np.random.rand(p,D,D))
    num.append(np.random.rand(p,D))
    return num

In [347]:
#создать две сопряженные друг другу MPS-цепочки
def create2MPSs(num):    
    loc1=[]
    loc2=[]
    for i in range(N):
        loc1.append(tn.Node(num[i]))
        loc2.append(tn.Node(np.conj(num[i])))
    
    loc1[0][1] ^ loc1[1][1]
    for i in range (1,N-1):
        loc1[i][2] ^ loc1[i+1][1]
    loc2[0][1] ^ loc2[1][1]
    for i in range (1,N-1):
        loc2[i][2] ^ loc2[i+1][1]
    
    return [loc1,loc2]

In [348]:
#кусок двойной цепочки PsiPsi превратить в 1 Node
def collapsepartPsiPsi(mps1,mps2,u,v):
    y=[]
    for i in range(u,v+1):
        y.append(mps1[i] @ mps2[i])
    z=shrink(y,v-u+1)
    return z

In [349]:
#кусок двойной цепочки PsiHPsi (с вставленными операторами между ними) превратить в 1 Node
def collapsepartPsiHPsi(mps1,mps2,s1,s2,k,u,v):
    y=[]
    if k!=N-1:
        g1=k
        g2=k+1
    else:
        g1=N-1
        g2=0
    for i in range(u,v+1):
        if i!=g1 and i!=g2:
            y.append(mps1[i] @ mps2[i])
        elif i==g1:
            y.append(mps1[i] @ s1 @ mps2[i])
        else:
            y.append(mps1[i] @ s2 @ mps2[i])
    z=shrink(y,v-u+1)
    return z

In [350]:
#сжать одну цепочку в 1 Node
def shrink(mps,length):
    if length!=0: #можно убрать, т.к. вызываем shrink только в таком случае
        res = mps[0]
        for i in range(1,length):
            res = res @ mps[i]
        return res

In [351]:
#создать Node с оператором sigma3 внутри
def createSigma3Node():
    sigma3node = tn.Node(np.array([[1,0],[0,-1]]))
    return sigma3node

In [352]:
#создать двойную цепочку с двумя соседними операторами sigma3 внутри (часть гамильтониана Изинга, только с другим знаком)
def createPsiHPsi(num,k):
    s1 = createSigma3Node()
    s2 = createSigma3Node()
    mpss=create2MPSs(num)
    mps1=mpss[0]
    mps2=mpss[1]
    if k!=N-1:
        g1=k
        g2=k+1
    else:
        g1=N-1
        g2=0
    for i in range(N):
        if i!=g1 and i!=g2:
            mps1[i][0] ^ mps2[i][0]
        elif i==g1:
            mps1[i][0] ^ s1[0]
            s1[1] ^ mps2[i][0]
        else:
            mps1[i][0] ^ s2[0]
            s2[1] ^ mps2[i][0]
    return [mps1,mps2,s1,s2]

In [353]:
#создать двойную цепочку PsiPsi
def createPsiPsi(num):
    mpss=create2MPSs(num)
    mps1=mpss[0]
    mps2=mpss[1]
    for i in range(N):
        mps1[i][0] ^ mps2[i][0]
    return [mps1,mps2]

In [354]:
#выделить из верхней цепочки вершину A, из нижней цепочки - вершину A*. и всё остальное. превратить это в [вектор, вектор, матрица]
def separateAfromPsiPsi(mps1,mps2,l):
    AA=mps1[l].tensor
    BB=mps2[l].tensor
    iden=np.identity(p)
    if l==0:
        r = collapsepartPsiPsi(mps1,mps2,1,N-1)
        bb=r.tensor
        return vecmatrveccorner(AA,BB,iden,bb)
    elif l==N-1:
        r = collapsepartPsiPsi(mps1,mps2,0,N-2)
        aa=r.tensor
        return vecmatrveccorner(AA,BB,iden,aa)
    else:
        r1 = collapsepartPsiPsi(mps1,mps2,0,l-1)
        r2 = collapsepartPsiPsi(mps1,mps2,l+1,N-1)
        aa=r1.tensor
        cc=r2.tensor
        return vecmatrvecmiddle(AA,BB,iden,aa,cc)

In [355]:
#выделить из верхней цепочки вершину A, из нижней цепочки - вершину A*. и всё остальное. превратить это в [вектор, вектор, матрица]
def separateAfromPsiHPsi(mps1,mps2,s1,s2,k,l):
    if k!=N-1:
        g1=k
        g2=k+1
    else:
        g1=N-1
        g2=0
    AA=mps1[l].tensor
    BB=mps2[l].tensor
    if g1==l:            
        op=s1.tensor
    elif g2==l:
        op=s2.tensor
    else:
        op=np.identity(p)
    
    if l==0:
        r = collapsepartPsiHPsi(mps1,mps2,s1,s2,k,1,N-1)
        bb=r.tensor
        return vecmatrveccorner(AA,BB,op,bb)
    elif l==N-1:
        r = collapsepartPsiHPsi(mps1,mps2,s1,s2,k,0,N-2)
        aa=r.tensor
        return vecmatrveccorner(AA,BB,op,aa)
    else:
        r1 = collapsepartPsiHPsi(mps1,mps2,s1,s2,k,0,l-1)
        r2 = collapsepartPsiHPsi(mps1,mps2,s1,s2,k,l+1,N-1)
        aa=r1.tensor
        cc=r2.tensor
        return vecmatrvecmiddle(AA,BB,op,aa,cc)

In [356]:
#превращение двух вершин и трех операторов между ними в [вектор, вектор, матрица]
def vecmatrvecmiddle(A,B,a0,a1,a2):
    q=D*p*D
    F = np.zeros(q)
    G = np.zeros((q,q))
    H = np.zeros(q)
    I=0
    J=0
    for i in range(p):
        for j in range(D):
            for k in range(D):
                F[I] = A[i][j][k]
                H[I] = B[i][j][k]
                for l in range(p):
                    for m in range(D):
                        for n in range(D):
                            G[I][J] = a0[i][l]*a1[j][m]*a2[k][n]
                            J=J+1
                I=I+1
                J=0
    return [F,G,H]

In [357]:
#превращение двух вершин и двух операторов между ними в [вектор, вектор, матрица] - происходит на краю цепочек
def vecmatrveccorner(A,B,a0,a1):
    q=p*D
    F = np.zeros(q)
    G = np.zeros((q,q))
    H = np.zeros(q)
    I=0
    J=0
    for i in range(p):
        for j in range(D):
            F[I] = A[i][j]
            H[I] = B[i][j]
            for l in range(p):
                for m in range(D):
                    G[I][J] = a0[i][l]*a1[j][m]
                    J=J+1
            I=I+1
            J=0
    return [F,G,H]

In [358]:
#получить Heff
def getHeff(m,l):
    if l==0 or l==N-1:
        q=p*D
    else:
        q=D*p*D
    G = np.zeros((q,q))
    for k in range(N):
        php=createPsiHPsi(m,k)
        t=separateAfromPsiHPsi(php[0],php[1],php[2],php[3],k,l)
        G += t[1]
    return -G

In [359]:
#получить N
def getNop(m,l):
    q=D*p*D
    pp=createPsiPsi(m)
    t=separateAfromPsiPsi(pp[0],pp[1],l)
    return [t[0],t[1]]

In [360]:
#превратить полученный вектор A в матрицу (чтобы затем ее вставить в цепочку вместо старого A)
def Avectomatr(vec,l):
    I=0
    if l==0 or l==N-1:
        matr = np.zeros((p,D))
        for i in range(p):
            for j in range(D):
                matr[i][j]=vec[I]
                I=I+1
    else:
        matr = np.zeros((p,D,D))
        for i in range(p):
            for j in range(D):
                for k in range(D):
                    matr[i][j][k]=vec[I]
                    I=I+1
    return matr

In [361]:
#посчитать энергию
def calcEnergy(num):
    E = 0
    for k in range(N):
        php=createPsiHPsi(num,k)
        t=collapsepartPsiHPsi(php[0],php[1],php[2],php[3],k,0,N-1)
        E -= t.tensor
    return E

In [362]:
N = 7 #длина цепочки
D = 3 #bond index
p = 2 #выходящий индекс
itnum = 1000 #число итераций для нахождения ground state

In [363]:
m=genrandarr()

In [364]:
for i in range(itnum):
    aloc=np.random.randint(N)
    e=getNop(m,aloc)
    A=e[0]
    Nop=e[1]
    Heff=getHeff(m,aloc)
    u=sp.linalg.eig(Heff,Nop)[1][0]
    v=Avectomatr(u,aloc)
    m[aloc]=v

/home/ekaterina/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [365]:
En = calcEnergy(m)
print(En)

-7.0
